In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

### Carregando o conjunto de dados para regressão. 

+ Vamos usar o conjunto de dados habitacional da Califórnia e criar um regressor com uma rede neural.

+ Depois de carregar os dados, dividimos em um conjunto de treinamento, um conjunto de validação e um conjunto de teste, e padronizamos todos os atributos.

In [3]:
# Baixa a base de dados.
housing = fetch_california_housing()

# Divide o conjunto total de exemplos em conjuntos de treinamento e teste.
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

# Divide o conjunto de treinamento em conjuntos de treinamento (menor) e validação.
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

# Aplica padronização às matrizes de atributos.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Diretório de logs

Em geral, apontamos o servidor do TensorBoard para um **diretório de log raiz** e configuramos o código para que ele grave em um **subdiretório diferente toda vez que for executado**. 

Dessa forma, a mesma instância do servidor do TensorBoard permitirá que você visualize e compare dados de várias execuções do seu código, sem misturá-los.

Portanto, começamos definindo o diretório de log raiz que usaremos para armazenar os logs do TensorBoard, além de uma função que gerará um caminho do subdiretório com base na data e hora atuais, para que ele seja diferente a cada execução. 

In [4]:
root_logdir = os.path.join(os.curdir, "my_logs")
print('Diretório raiz:', root_logdir)

def get_run_logdir(root_logdir):
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir(root_logdir)
print('Subdiretório raiz:', run_logdir)

Diretório raiz: .\my_logs
Subdiretório raiz: .\my_logs\run_2023_06_14-22_58_51


### Criando e compilando o modelo.

In [5]:
model = keras.models.Sequential(
    [
        keras.layers.Dense(30, activation="relu", input_shape=[8], name='hidden1'),
        keras.layers.Dense(30, activation="relu", name='hidden2'),
        keras.layers.Dense(1, name='output')
    ]
)

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-4))

### Tensorboard callback

O Keras fornece uma callback chamada **TensorBoard**, a qual passamos o caminho do **subdiretório** através do parâmetro `log_dir`.


O parâmetro `histogram_freq` configura a frequência (em épocas) em que os histogramas dos pesos das camadas do modelo são calculados.

In [6]:
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=run_logdir, histogram_freq=1)

### Treinando o modelo.

In [7]:
history = model.fit(X_train, y_train, 
                    epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb]
                   )

Epoch 1/30
363/363 [==============================] - 2s 2ms/step - loss: 3.6085 - val_loss: 7.5774
Epoch 2/30
363/363 [==============================] - 1s 2ms/step - loss: 2.6045 - val_loss: 9.5229
Epoch 3/30
363/363 [==============================] - 1s 2ms/step - loss: 2.0247 - val_loss: 9.8846
Epoch 4/30
363/363 [==============================] - 1s 2ms/step - loss: 1.6750 - val_loss: 8.9279
Epoch 5/30
363/363 [==============================] - 1s 2ms/step - loss: 1.4462 - val_loss: 7.6330
Epoch 6/30
363/363 [==============================] - 1s 2ms/step - loss: 1.2843 - val_loss: 6.6214
Epoch 7/30
363/363 [==============================] - 1s 2ms/step - loss: 1.1650 - val_loss: 5.5855
Epoch 8/30
363/363 [==============================] - 1s 2ms/step - loss: 1.0741 - val_loss: 4.5773
Epoch 9/30
363/363 [==============================] - 1s 2ms/step - loss: 1.0023 - val_loss: 3.7520
Epoch 10/30
363/363 [==============================] - 1s 2ms/step - loss: 0.9443 - val_loss: 3.0705

### Inicializando o servidor do Tensorboard

Em seguida, executamos o seguinte comando na raiz do projeto (ou de qualquer outro lugar, desde que apontemos para o diretório de log apropriado).

Para iniciar o servidor TensorBoard, uma opção é abrir um terminal. Em seguida, acessamos o diretório deste notebook e digitamos:

`$ tensorboard --logdir=./my_logs --port=6006`

Na sequência, abrimos o navegador web e o apontamos para `localhost:6006` e usamos o TensorBoard.

Como alternativa, podemos carregar a extensão Jupyter do TensorBoard e executá-lo como mostrado abaixo.

In [8]:
%load_ext tensorboard

In [9]:
%tensorboard --logdir=./my_logs --port=6006

#### Observações.

+ Além do erro real, o tensorboard mostra uma versão suavizada do erro (**smoothed**), onde se aplica uma **média móvel exponencialmente descrescente** aos valores do erro.
    + A suavização ajuda a analisar a tendência do erro quando as atualizações são muito ruidosas.


+ O **histograma** dos pesos das camadas mostra no eixo x o intervalo de valores dos pesos, no eixo y, a ocorrência normalizada dos valores e no eixo z, e época.

### Executando o modelo com valor maior para o passo de aprendizagem.

In [10]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [11]:
run_logdir2 = get_run_logdir(root_logdir)
print('Subdiretório raiz:', run_logdir2)

Subdiretório raiz: .\my_logs\run_2023_06_14-22_59_20


In [12]:
model = keras.models.Sequential(
    [
        keras.layers.Dense(30, activation="relu", input_shape=[8], name='hidden1'),
        keras.layers.Dense(30, activation="relu", name='hidden2'),
        keras.layers.Dense(1, name='output')
    ]
)

model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [13]:
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=run_logdir2, histogram_freq=1)

history = model.fit(X_train, y_train, 
                    epochs=30,
                    validation_data=(X_valid, y_valid),
                    callbacks=[tensorboard_cb])

Epoch 1/30
363/363 [==============================] - 1s 2ms/step - loss: 1.9040 - val_loss: 0.9000
Epoch 2/30
363/363 [==============================] - 1s 2ms/step - loss: 0.8061 - val_loss: 0.7393
Epoch 3/30
363/363 [==============================] - 1s 2ms/step - loss: 0.7144 - val_loss: 0.6543
Epoch 4/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6683 - val_loss: 0.6760
Epoch 5/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6326 - val_loss: 0.6377
Epoch 6/30
363/363 [==============================] - 1s 2ms/step - loss: 0.6010 - val_loss: 0.6279
Epoch 7/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5743 - val_loss: 0.5825
Epoch 8/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5500 - val_loss: 0.5698
Epoch 9/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5282 - val_loss: 0.5600
Epoch 10/30
363/363 [==============================] - 1s 2ms/step - loss: 0.5091 - val_loss: 0.5496

+ Observe como o TensorBoard agora vê duas execuções e podemos comparar as curvas de aprendizado.

In [14]:
%tensorboard --logdir=./my_logs --port=6006

Reusing TensorBoard on port 6006 (pid 17048), started 0:00:21 ago. (Use '!kill 17048' to kill it.)

### Observação

+ Observem que a perda de treinamento caiu bastante durante as duas execuções, mas a segunda foi muito mais rápida. 


+ Isso se deve ao fato de termos usado uma taxa de aprendizado maior.

### Referências

[1] https://towardsdatascience.com/a-quickstart-guide-to-tensorboard-fb1ade69bbcf

[2] https://neptune.ai/blog/tensorboard-tutorial